# Spatio-Temporal Denoising Graph Autoencoder (STD-GAE) 

STD-GAE exploits temporal correlation, spatial coherence and value dependencies from domain knowledge to recover missing data. It is empowered by two modules. 

1.   To cope with sparse yet various scenarios of missing data, STD-GAE incorporates a domain-knowledge aware data augmentation module
that creates plausible variations of missing data patterns. This generalizes
STD-GAE to robust imputation over different seasons and
environment.
2.   STD-GAE nontrivially integrates spatiotemporal
graph convolution layers (to recover local missing data by observed
“neighboring” PV plants) and denoising autoencoder (to recover
corrupted data from augmented counterpart) to improve the accu-
racy of imputation accuracy at PV fleet level. 



In [1]:
import sys
sys.path = ['/home/axn392/gnn',
 '',
 '/usr/local/lib/python3.8/dist-packages',
 '/home/rxf131/ondemand/ubuntu2004/torch-geometric-temporal',
 '/usr/lib/python38.zip',
 '/usr/lib/python3.8',
 '/usr/lib/python3.8/lib-dynload',
 '/usr/lib/python3/dist-packages']
sys.path

['/home/axn392/gnn',
 '',
 '/usr/local/lib/python3.8/dist-packages',
 '/home/rxf131/ondemand/ubuntu2004/torch-geometric-temporal',
 '/usr/lib/python38.zip',
 '/usr/lib/python3.8',
 '/usr/lib/python3.8/lib-dynload',
 '/usr/lib/python3/dist-packages']

In [2]:
from datetime import datetime
#import geopy.distance # to compute distances between stations
import glob
import numpy as np
import os
import pandas as pd
import scipy.sparse as sp
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
#from torch_geometric_temporal.nn import STConv
from tqdm import tqdm
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import ChebConv

/home/rxf131/ondemand/ubuntu2004/torch-geometric-temporal/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#Generate Edge Weight Matrix 

We represent the spatiotemporal PV data as an undirected graph $G = (V, E, X_{t})$. 

1.   Each node in $V$ represents a PV inverter
2.   Edges $E$ are assigned according to Edge Weight Matrix (if $W_{i,j}$ > 0, then there is edge between i and j)
3.   $X_{t}$ denotes a node attribute tensor $\in \mathbb{R}^{T\times n\times d}$. Here T is the length of timeseries, $n$ is the number of nodes (which is 98 in our study), and $d$ is the number of input channel.

Since the locations of PV inverters are fixed, the graph structure is static with time-invariant nodes and edges. 
However, $X_{t}$ is time-varying: each node i carries a  timeseries $x_{i} \in \mathbb{R}^{T\times d}$ recording attributes 
such as temperature, wind speed, 
irradiance and power output.

In [3]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

In [4]:
import pandas as pd
import numpy as np

# please change the path according to your setting
location = pd.read_csv('location_35.csv',index_col=0)
distance = np.zeros(shape=(35,35))
dist = []
for i in range(35):
    for j in range(35):
        d = haversine(location.iloc[i][1], location.iloc[i][0], location.iloc[j][1], location.iloc[j][0])
        distance[i][j] = d
        dist.append(d)

dist_std = np.std(dist)
distance = pd.DataFrame(distance)
distance

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698,10.142698


In the next step, we compute $w_{ij}$ for each pair of nodes $(i, j)$ in the graph as presented in Yu et al. (2018):

$W_{i,j} = \begin{cases} \exp{(-\frac{d_{ij}^2}{\sigma^2})}, i\neq j \text{ and } \exp{(-\frac{d_{ij}^2}{\sigma^2})} \geq \epsilon \\ 
0 \text{ otherwise }
\end{cases}$

In this case, $d_{ij}$ for each pair of nodes $(i, j)$ is the distance between stations that we have computed above. $\sigma$ is the standard deviation of the distances.

In [5]:
# epsilon = 0, 0.25, 0.5, 0.75, 1
epsilon = 1
sigma = dist_std
W = np.zeros(shape=(35,35))

for i in range(35):
    for j in range(35):
        if i == j: 
            W[i][j] = 0
        else:
            # Compute distance between stations
            d_ij = distance.loc[i][j]
            
            # Compute weight w_ij
            w_ij = np.exp(-d_ij**2 / sigma**2)
            
            if w_ij >= epsilon:
                W[i, j] = w_ij

W = pd.DataFrame(W)
# please change the path according to your setting
W.to_csv('W_35.csv',index=False)

#Construction of STD-GAE 


*   Construct temporal & spatial layers -> ST Blocks (encoder & decoder) -> Graph Denoising Autoencoders 
*   Define Parameters
*   Some utility functions


Construction of STD-GAE is the foundation for later codes.

Temporal Convolutional (TConv) Layers and Deconvolutional (DeConv) Layers

In [6]:
class TemporalConv(nn.Module):

    """
    Args:
        in_channels (int): Number of input features.
        out_channels (int): Number of output features.
        kernel_size (int): Convolutional kernel size.
    """

    def __init__(self, in_channels: int, out_channels: int, kernel_size, stride: int, padding: int):
        super(TemporalConv, self).__init__()
        self.conv_1 = nn.Conv2d(in_channels, out_channels, (1, kernel_size), (1, stride), (0,padding))
        self.conv_2 = nn.Conv2d(in_channels, out_channels, (1, kernel_size), (1, stride), (0,padding))
        self.conv_3 = nn.Conv2d(in_channels, out_channels, (1, kernel_size), (1, stride), (0,padding))

    def forward(self, X: torch.FloatTensor) -> torch.FloatTensor:
        """Forward pass through temporal convolution block.

        Arg types:
            * **X** (torch.FloatTensor) -  Input data of shape
                (batch_size, input_time_steps, num_nodes, in_channels).

        Return types:
            * **H** (torch.FloatTensor) - Output data of shape
                (batch_size, in_channels, num_nodes, input_time_steps).
        """
        X = X.permute(0, 3, 2, 1)
        P = self.conv_1(X)
        Q = torch.sigmoid(self.conv_2(X))
        PQ = P * Q
        H = F.relu(PQ + self.conv_3(X))
        H = H.permute(0, 3, 2, 1)
        return H

class TemporalDeConv1(nn.Module):

    """
    Args:
        in_channels (int): Number of input features.
        out_channels (int): Number of output features.
        kernel_size (int): Convolutional kernel size.
    """

    def __init__(self, in_channels: int, out_channels: int, kernel_size, stride: int, padding: int):
        super(TemporalDeConv1, self).__init__()
        self.conv_1 = nn.ConvTranspose2d(in_channels, out_channels, (1, kernel_size), (1, stride), (0,padding))
        self.conv_2 = nn.ConvTranspose2d(in_channels, out_channels, (1, kernel_size),(1, stride), (0,padding))
        self.conv_3 = nn.ConvTranspose2d(in_channels, out_channels, (1, kernel_size),(1, stride), (0,padding))

    def forward(self, X: torch.FloatTensor) -> torch.FloatTensor:
        """Forward pass through temporal convolution block.

        Arg types:
            * **X** (torch.FloatTensor) -  Input data of shape
                (batch_size, input_time_steps, num_nodes, in_channels).

        Return types:
            * **H** (torch.FloatTensor) - Output data of shape
                (batch_size, in_channels, num_nodes, input_time_steps).
        """
        X = X.permute(0, 3, 2, 1)
        P = self.conv_1(X)
        Q = torch.sigmoid(self.conv_2(X))
        PQ = P * Q
        H = F.relu(PQ + self.conv_3(X))
        H = H.permute(0, 3, 2, 1)
        return H

class TemporalDeConv2(nn.Module):

    """
    Args:
        in_channels (int): Number of input features.
        out_channels (int): Number of output features.
        kernel_size (int): Convolutional kernel size.
    """

    def __init__(self, in_channels: int, out_channels: int, kernel_size, stride: int):
        super(TemporalDeConv2, self).__init__()
        self.conv_1 = nn.ConvTranspose2d(in_channels, out_channels, (1, kernel_size),(1, stride))
        self.conv_2 = nn.ConvTranspose2d(in_channels, out_channels, (1, kernel_size),(1, stride))
        self.conv_3 = nn.ConvTranspose2d(in_channels, out_channels, (1, kernel_size),(1, stride))

    def forward(self, X: torch.FloatTensor) -> torch.FloatTensor:
        """Forward pass through temporal convolution block.

        Arg types:
            * **X** (torch.FloatTensor) -  Input data of shape
                (batch_size, input_time_steps, num_nodes, in_channels).

        Return types:
            * **H** (torch.FloatTensor) - Output data of shape
                (batch_size, in_channels, num_nodes, input_time_steps).
        """
        X = X.permute(0, 3, 2, 1)
        P = self.conv_1(X)
        Q = torch.sigmoid(self.conv_2(X))
        PQ = P * Q
        H = F.relu(PQ + self.conv_3(X))
        H = H.permute(0, 3, 2, 1)
        return H


Encoder: a Spaio-temporal Block (Temporal Conv + Spatial Conv + Temporal Conv)

In [7]:
class STConvEncoder(nn.Module):

    def __init__(
        self,
        num_nodes: int,
        in_channels: int,
        hidden_channels: int,
        out_channels: int,
        kernel_size: int,
        stride: int,
        padding: int,
        K: int,
        normalization: str = "sym",
        bias: bool = True,
    ):
        super(STConvEncoder, self).__init__()
        self.num_nodes = num_nodes
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.K = K
        self.normalization = normalization
        self.bias = bias

        self._temporal_conv1 = TemporalConv(
            in_channels=in_channels,
            out_channels=hidden_channels,
            kernel_size=kernel_size, stride = stride, padding = padding,
        )

        self._graph_conv = ChebConv(
            in_channels=hidden_channels,
            out_channels=hidden_channels,
            K=K,
            normalization=normalization,
            bias=bias,
        )

        self._temporal_conv2 = TemporalConv(
            in_channels=hidden_channels,
            out_channels=out_channels,
            kernel_size=kernel_size, stride = stride, padding = padding,
        )

        self._batch_norm = nn.BatchNorm2d(num_nodes)

    def forward(self, X: torch.FloatTensor, edge_index: torch.LongTensor, edge_weight: torch.FloatTensor = None,) -> torch.FloatTensor:

        r"""Forward pass. If edge weights are not present the forward pass
        defaults to an unweighted graph.

        Arg types:
            * **X** (PyTorch FloatTensor) - Sequence of node features of shape (Batch size X Input time steps X Num nodes X In channels).
            * **edge_index** (PyTorch LongTensor) - Graph edge indices.
            * **edge_weight** (PyTorch LongTensor, optional)- Edge weight vector.

        Return types:
            * **T** (PyTorch FloatTensor) - Sequence of node features.
        """
        #print(X.shape)
        T_0 = self._temporal_conv1(X)
        #print(T_0.shape)
        T = torch.zeros_like(T_0).to(T_0.device)
        for b in range(T_0.size(0)):
            for t in range(T_0.size(1)):
                T[b][t] = self._graph_conv(T_0[b][t], edge_index, edge_weight)

        T = F.relu(T)
        #print(T.shape)
        T = self._temporal_conv2(T)
        #print(T.shape)
        # T = T.permute(0, 2, 1, 3)
        # #print(T.shape)
        # T = self._batch_norm(T)
        # T = T.permute(0, 2, 1, 3)
        #print(T.shape)

        return T

Decoder: a Spaio-temporal Block (Temporal DeConv + Spatial Conv + Temporal DeConv)

In [8]:
class STConvDecoder(nn.Module):

    def __init__(
        self,
        num_nodes: int,
        in_channels: int,
        hidden_channels: int,
        out_channels: int,
        kernel_size: int,
        kernel_size_de: int,
        stride: int,
        padding: int,
        K: int,
        normalization: str = "sym",
        bias: bool = True,
    ):
        super(STConvDecoder, self).__init__()
        self.num_nodes = num_nodes
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.K = K
        self.normalization = normalization
        self.bias = bias

        self._temporal_conv1 = TemporalDeConv1(
            in_channels=in_channels,
            out_channels=hidden_channels,
            kernel_size=kernel_size, stride = stride, padding = padding,
        )

        self._graph_conv = ChebConv(
            in_channels=hidden_channels,
            out_channels=hidden_channels,
            K=K,
            normalization=normalization,
            bias=bias,
        )

        self._temporal_conv2 = TemporalDeConv2(
            in_channels=hidden_channels,
            out_channels=out_channels,
            kernel_size=kernel_size_de, stride = stride,
        )

        self._batch_norm = nn.BatchNorm2d(num_nodes)

    def forward(self, X: torch.FloatTensor, edge_index: torch.LongTensor, edge_weight: torch.FloatTensor = None,) -> torch.FloatTensor:

        r"""Forward pass. If edge weights are not present the forward pass
        defaults to an unweighted graph.

        Arg types:
            * **X** (PyTorch FloatTensor) - Sequence of node features of shape (Batch size X Input time steps X Num nodes X In channels).
            * **edge_index** (PyTorch LongTensor) - Graph edge indices.
            * **edge_weight** (PyTorch LongTensor, optional)- Edge weight vector.

        Return types:
            * **T** (PyTorch FloatTensor) - Sequence of node features.
        """
        T_0 = self._temporal_conv1(X)
        T = torch.zeros_like(T_0).to(T_0.device)
        for b in range(T_0.size(0)):
            for t in range(T_0.size(1)):
                T[b][t] = self._graph_conv(T_0[b][t], edge_index, edge_weight)

        T = F.relu(T)
        T = self._temporal_conv2(T)
        # T = T.permute(0, 2, 1, 3)
        # T = self._batch_norm(T)
        # T = T.permute(0, 2, 1, 3)
        return T

Denoising Graph Autoencoder

In [9]:
# a specified number of STConv blocks, followed by an output layer
class STConvAE(torch.nn.Module):
    def __init__(self, device, num_nodes, channel_size_list, num_layers, 
                 kernel_size, K, window_size, kernel_size_de, stride, padding,\
                 normalization = 'sym', bias = True):
    # num_nodes = number of nodes in the input graph
    # channel_size_list =  2d array representing feature dimensions throughout the model
    # num_layers = number of STConv blocks
    # kernel_size = length of the temporal kernel
    # K = size of the chebyshev filter for the spatial convolution
    # window_size = number of historical time steps to consider

        super(STConvAE, self).__init__()
        self.layers = nn.ModuleList([])
        # add STConv blocks
        for l in range(num_layers):
            input_size, hidden_size, output_size = channel_size_list[l][0], channel_size_list[l][1], channel_size_list[l][2]
            if l==0:
                self.layers.append(STConvEncoder(num_nodes, input_size, hidden_size, output_size, kernel_size, stride, padding, K, normalization, bias))
            if l==1:
                self.layers.append(STConvDecoder(num_nodes, input_size, hidden_size, output_size, kernel_size, kernel_size_de, stride, padding, K, normalization, bias))
        

        # # add output layer
        # self.layers.append(OutputLayer(channel_size_list[-1][-1], \
        #                                window_size - 2 * num_layers * (kernel_size - 1), \
        #                                num_nodes))
        # CUDA if available
        for layer in self.layers:
            layer = layer.to(device)

    def forward(self, x, edge_index, edge_weight ):
        #print(x.shape)
        for layer in self.layers:
            x = layer(x, edge_index, edge_weight)
          #print(x.shape)
        # out_layer = self.layers[-1]
        # x = x.permute(0, 3, 1, 2)
        # x = out_layer(x)
        # print(x.shape)
        return x

Define Parameters

In [10]:
# model parameters
num_nodes = 35
#channels = np.array([[1, 1, 1], [1, 1, 1]]) # sequence of channel sizes
channels = np.array([[1, 8, 16], [16, 8, 1]])
kernel_size = 4 # size of temporal kernel
kernel_size_de = 2 # size of temporal deconv2
stride = 2
padding = 1
K = 3 # chebyshev filter size

# training parameters
learning_rate = 0.001
batch_size = 2
num_epochs = 50 # note that we trained for 7 epochs using Google Cloud
num_layers = 2 # number of STConv blocks
n_his = 288 # window size
train_prop = 2/3 # Our actual training set proportion was 0.7
val_prop = 1/6 # Our actual training set proportion was 0.2
test_prop = 1/6 # Our actual training set proportion was 0.1

# model save path
model_save_path = os.path.join('best_model_12hr_BM.pt')

Preparing Data

In [11]:
def data_transform(data, corrupted_data, window, device):
    # data = slice of V matrix
    # n_his = number of historical speed observations to consider
    # n_pred = number of time steps in the future to predict

    num_nodes = data.shape[1]
    num_obs = int(len(data)/window)
    x = np.zeros([num_obs, window, num_nodes, 1])
    y = np.zeros([num_obs, window, num_nodes, 1])
    
    obs_idx = 0
    for i in range(num_obs):
        head = i*window
        tail = (i+1)*window
        y[obs_idx, :, :, :] = data[head: tail].reshape(n_his, num_nodes, 1)
        x[obs_idx, :, :, :] = corrupted_data[head: tail].reshape(n_his, num_nodes, 1)
        #x[obs_idx, :, :, :] = data[head: tail].reshape(n_his, num_nodes, 1)
        obs_idx += 1

    return torch.Tensor(x).to(device), torch.Tensor(y).to(device)

#STD-GAE Framework
STD-GAE framework consists of the following four major components: data ingestion, data augmentation, data corruption, and STD-GAE.

#Data Ingestion

In [12]:
# please change the path according to your setting
W = pd.read_csv('W_35.csv')
D_O = pd.read_csv('norm_power_35.csv')
D_O

,0,1,2,3,4,5,6,7,8,9,...,103670,103671,103672,103673,103674,103675,103676,103677,103678,103679
0,25.612245,23.771429,18.341497,3.925170,4.761905,15.587755,16.447619,10.442177,12.428571,6.717007,...,43.736054,42.488435,41.051701,39.389116,37.186395,35.801361,33.794558,18.436735,28.198639,14.804082
1,25.749660,23.971429,18.451701,3.868027,4.624490,15.868027,16.703401,10.451701,12.578231,6.692517,...,44.696599,43.454422,41.942857,40.287075,38.038095,36.454422,34.304762,18.800000,28.595918,15.008163
2,25.725170,24.070748,18.303401,3.914286,4.012245,16.587755,16.337415,10.853061,12.319728,6.722449,...,44.721088,43.390476,41.944218,40.225850,38.035374,36.477551,34.224490,18.157823,29.416327,15.093878
3,26.780952,25.200000,19.444898,3.891156,4.248980,17.557823,18.242177,10.834014,13.568707,6.640816,...,43.934694,42.791837,41.670748,39.816327,37.857143,36.507483,34.556463,17.353741,30.627211,15.142857
4,26.457143,24.828571,19.061224,3.869388,4.195918,17.287075,17.944218,10.682993,13.353741,6.696599,...,43.980952,42.767347,41.289796,39.642177,37.447619,36.341497,34.293878,17.238095,30.289796,15.117007
5,25.548299,23.785034,18.357823,3.948299,5.053061,15.444898,16.398639,10.702041,12.380952,6.741497,...,44.436735,42.955102,41.545578,39.817687,37.710204,36.108844,33.960544,18.186395,28.808163,14.918367
6,26.268027,24.627211,18.546939,3.838095,4.099320,16.644898,16.937415,10.206803,12.495238,6.786395,...,42.507483,35.410884,26.703401,19.514286,13.745578,11.927891,10.303401,7.288435,9.053061,7.529252
7,24.129252,22.546939,17.331973,3.643537,4.127891,14.978231,15.704762,9.718367,11.963265,6.187755,...,42.770068,41.612245,40.208163,38.642177,36.843537,35.292517,33.368707,16.865306,29.711565,14.817687
8,25.778231,23.959184,18.512925,3.956463,4.779592,15.725170,16.526531,10.544218,12.523810,6.729252,...,44.179592,42.805442,41.390476,39.629932,37.440816,36.023129,34.048980,18.526531,28.393197,14.982313
9,25.840816,24.195918,18.447619,3.881633,4.093878,16.447619,16.834014,10.232653,12.440816,6.797279,...,44.839456,43.468027,41.910204,40.216327,37.910204,36.499320,34.387755,17.778231,29.835374,15.106122


#Data Augmentation

In [13]:
from sklearn.impute import KNNImputer

D_O = D_O.T
imputer = KNNImputer(n_neighbors=5)
D_O = pd.DataFrame(D_O)
imputer.fit(D_O)
D_A = imputer.transform(D_O)
D_A = pd.DataFrame(D_A)
D_A.to_csv('Data_Augmented.csv', index = False) 
D_A

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,25.612245,25.749660,25.725170,26.780952,26.457143,25.548299,26.268027,24.129252,25.778231,25.840816,...,23.055782,23.639456,22.740136,26.885714,26.553741,22.427211,26.444898,25.571429,26.088435,22.616327
1,23.771429,23.971429,24.070748,25.200000,24.828571,23.785034,24.627211,22.546939,23.959184,24.195918,...,21.994558,22.400000,21.610884,25.025850,24.657143,21.193197,24.634014,23.828571,24.277551,21.551020
2,18.341497,18.451701,18.303401,19.444898,19.061224,18.357823,18.546939,17.331973,18.512925,18.447619,...,20.797279,21.208163,20.442177,23.257143,22.868027,20.039456,22.876190,22.189116,22.595918,20.451701
3,3.925170,3.868027,3.914286,3.891156,3.869388,3.948299,3.838095,3.643537,3.956463,3.881633,...,19.537415,19.953741,19.206803,21.571429,21.076190,18.765986,21.085714,20.408163,20.794558,19.082993
4,4.761905,4.624490,4.012245,4.248980,4.195918,5.053061,4.099320,4.127891,4.779592,4.093878,...,18.379592,18.775510,18.013605,19.989116,19.401361,17.681633,19.461224,18.786395,19.208163,18.013605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103675,35.801361,36.454422,36.477551,36.507483,36.341497,36.108844,11.927891,35.292517,36.023129,36.499320,...,34.114286,35.053061,33.802721,38.580952,38.734557,33.336054,38.152381,36.774150,37.485714,33.412245
103676,33.794558,34.304762,34.224490,34.556463,34.293878,33.960544,10.303401,33.368707,34.048980,34.387755,...,31.717007,32.553741,31.421769,35.375510,35.371429,31.043537,35.178231,33.546939,34.114286,31.248980
103677,18.436735,18.800000,18.157823,17.353741,17.238095,18.186395,7.288435,16.865306,18.526531,17.778231,...,21.776871,22.152381,21.410884,22.733333,23.401361,21.114286,22.476190,21.897959,22.127891,20.971429
103678,28.198639,28.595918,29.416327,30.627211,30.289796,28.808163,9.053061,29.711565,28.393197,29.835374,...,7.672109,7.926531,7.600000,6.402721,7.118367,7.760544,6.949660,6.336054,5.922449,7.627211


#Data Corruption

We provide the choice of 12 different missing masks to corrupt, which includes:


1.   Type I: Missing Completely at Random (MCAR): 10%, 20%, 30%, 40%, 50%, and 60%
2.   Type II: Block Missing (BM): 2hrs, 4hrs, 6hrs, 8hrs, 10hrs, and 12hrs.

The missing maks can be adjusted if the prior distribution of missing data is given.



MCAR Masks

Please change the path according to your setting.

In [ ]:
mask = torch.FloatTensor(103680, 98).uniform_() > 0.2
pd.DataFrame(mask.numpy()).to_csv('gdrive/My Drive/STGCN/20%MCAR.csv',index=False)

In [ ]:
mask = torch.FloatTensor(103680, 98).uniform_() > 0.3
pd.DataFrame(mask.numpy()).to_csv('gdrive/My Drive/STGCN/30%MCAR.csv',index=False)

In [ ]:
mask = torch.FloatTensor(103680, 98).uniform_() > 0.4
pd.DataFrame(mask.numpy()).to_csv('gdrive/My Drive/STGCN/40%MCAR.csv',index=False)

In [ ]:
mask = torch.FloatTensor(103680, 98).uniform_() > 0.5
pd.DataFrame(mask.numpy()).to_csv('gdrive/My Drive/STGCN/50%MCAR.csv',index=False)

In [ ]:
mask = torch.FloatTensor(103680, 98).uniform_() > 0.6
pd.DataFrame(mask.numpy()).to_csv('gdrive/My Drive/STGCN/60%MCAR.csv',index=False)

BM Masks
Please change the path according to your setting.

In [ ]:
import random

length=24
mask = torch.full((103680,98), True)
for i in range(360):
    for j in range(98):
        number = random.randint(288*i+1,288*(i+1)-length-1)
        mask[number:number+length, j] = False

pd.DataFrame(mask.numpy()).to_csv('gdrive/My Drive/STGCN/2hr_BM.csv',index=False)

In [14]:
import random

length=48
mask = torch.full((103680,35), True)
for i in range(360):
    for j in range(35):
        number = random.randint(288*i+1,288*(i+1)-length-1)
        mask[number:number+length, j] = False

pd.DataFrame(mask.numpy()).to_csv('4hr_BM.csv',index=False)

In [15]:
import random

length=72
mask = torch.full((103680,35), True)
for i in range(360):
    for j in range(35):
        number = random.randint(288*i+1,288*(i+1)-length-1)
        mask[number:number+length, j] = False

pd.DataFrame(mask.numpy()).to_csv('6hr_BM.csv',index=False)

In [ ]:
import random

length=96
mask = torch.full((103680,98), True)
for i in range(360):
    for j in range(98):
        number = random.randint(288*i+1,288*(i+1)-length-1)
        mask[number:number+length, j] = False

pd.DataFrame(mask.numpy()).to_csv('gdrive/My Drive/STGCN/8hr_BM.csv',index=False)

In [ ]:
import random

length=120
mask = torch.full((103680,98), True)
for i in range(360):
    for j in range(98):
        number = random.randint(288*i+1,288*(i+1)-length-1)
        mask[number:number+length, j] = False

pd.DataFrame(mask.numpy()).to_csv('gdrive/My Drive/STGCN/10hr_BM.csv',index=False)

In [15]:
import random

length=144
mask = torch.full((103680,35), True)
for i in range(360):
    for j in range(35):
        number = random.randint(288*i+1,288*(i+1)-length-1)
        mask[number:number+length, j] = False

pd.DataFrame(mask.numpy()).to_csv('12hr_BM.csv',index=False)

In [16]:
#Choose the mask you want to corrupt D_A: here we choose 12hrs BM
mask = pd.read_csv('12hr_BM.csv')
mask = torch.tensor(mask.values)
D_C = pd.read_csv('Data_Augmented.csv')
D_C[mask.numpy()==False] = -1
D_A = pd.read_csv('Data_Augmented.csv')

#STD-GAE Training

Data Preprocessing

In [17]:
power_tensor = torch.tensor(D_A.values)
length = D_A.shape[0]
train_x = power_tensor[0:int(train_prop*length),:].to(torch.float32)
validation_x = power_tensor[int(train_prop*length):int((train_prop+val_prop)*length)+1,:].to(torch.float32)
test_x = power_tensor[int((train_prop+val_prop)*length)+1:length,:].to(torch.float32)

power_corrupted_tensor = torch.tensor(D_C.values)
length = D_C.shape[0]
corrupted_train_x = power_corrupted_tensor[0:int(train_prop*length),:].to(torch.float32)
corrupted_validation_x = power_corrupted_tensor[int(train_prop*length):int((train_prop+val_prop)*length)+1,:].to(torch.float32)
corrupted_test_x = power_corrupted_tensor[int((train_prop+val_prop)*length)+1:length,:].to(torch.float32)

In [18]:
device = torch.device("cuda") if torch.cuda.is_available() \
else torch.device("cpu")

x_train, y_train = data_transform(train_x.numpy(), corrupted_train_x.numpy(), n_his, device)
x_val, y_val = data_transform(validation_x.numpy(), corrupted_validation_x.numpy(), n_his, device)
x_test, y_test = data_transform(test_x.numpy(), corrupted_test_x.numpy(), n_his, device)

# create torch data iterables for training
train_data = torch.utils.data.TensorDataset(x_train, y_train)
train_iter = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True)
val_data = torch.utils.data.TensorDataset(x_val, y_val)
val_iter = torch.utils.data.DataLoader(val_data, batch_size)
test_data = torch.utils.data.TensorDataset(x_test, y_test)
test_iter = torch.utils.data.DataLoader(test_data, batch_size)

# format graph for pyg layer inputs
G = sp.coo_matrix(W)
edge_index = torch.tensor(np.array([G.row, G.col]), dtype=torch.int64).to(device)
edge_weight = torch.tensor(G.data).float().to(device)

Model Training

In [19]:
model = STConvAE(device, num_nodes, channels, num_layers, kernel_size, K, n_his, kernel_size_de, stride, padding, normalization = 'sym', bias = True).to(device)
# define loss function
loss = nn.MSELoss()
# define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay = 0.02) 

In [ ]:
min_valid_loss = np.inf

for epoch in tqdm(range(1, num_epochs + 1), desc = 'Epoch', position = 0):
    train_loss, n = 0.0, 0
    model.train()
    
    for x, y in tqdm(train_iter, desc = 'Batch', position = 0):
        # get model predictions and compute loss
        y_pred = model(x.to(device), edge_index, edge_weight)
        loss = torch.mean((y_pred-y)**2)
        # backpropogation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    valid_loss = 0.0
    model.eval() 
    for x, y in tqdm(val_iter, desc = 'Batch', position = 0):
        # get model predictions and compute loss
        y_pred = model(x.to(device), edge_index, edge_weight)
        loss = torch.mean((y_pred-y)**2)
        valid_loss += loss.item() 

    print(f'Epoch {epoch} \t\t Training Loss: {train_loss/120} \t\t Validation Loss: {valid_loss/30}')
    if min_valid_loss > valid_loss:
        min_valid_loss = valid_loss
        torch.save(model.state_dict(), model_save_path)

Epoch:   2%|▏         | 1/50 [01:54<1:33:27, 114.45s/it]

Epoch 1 		 Training Loss: 321.651522954305 		 Validation Loss: 86.71108996073404


Epoch:   4%|▍         | 2/50 [03:48<1:31:24, 114.27s/it]

Epoch 2 		 Training Loss: 66.79544210036596 		 Validation Loss: 41.32528076171875


Epoch:   6%|▌         | 3/50 [05:43<1:29:53, 114.75s/it]

...

Epoch:  90%|█████████ | 45/50 [1:25:45<09:30, 114.17s/it]

Epoch 45 		 Training Loss: 23.70346832672755 		 Validation Loss: 9.121953582763672


Epoch:  94%|█████████▍| 47/50 [1:29:34<05:42, 114.20s/it]

Epoch 47 		 Training Loss: 23.773285833994546 		 Validation Loss: 10.436311308542887


Batch:  58%|█████▊    | 70/120 [00:56<00:39,  1.26it/s]

Evaluate Model

In [21]:
# load model with lowest validation lost
best_model = STConvAE(device, num_nodes, channels, num_layers, kernel_size, K, n_his, kernel_size_de, stride, padding, normalization = 'sym', bias = True).to(device)
best_model.load_state_dict(torch.load(model_save_path))

best_model.eval()
cost = 0
missing_count = 0
predicted = []
ground_truth = []

i = 1

for x, y in tqdm(test_iter, desc = 'Batch', position = 0):
    # get model predictions and compute loss
    y_pred = best_model(x.to(device), edge_index, edge_weight)
    if i == 1:
        y_pred_complete = y_pred
    else:
        y_pred_complete = torch.cat((y_pred_complete, y_pred), 0)
    i+=1

print(y_pred_complete.shape)

Batch: 100%|██████████| 30/30 [00:23<00:00,  1.30it/s]

torch.Size([60, 288, 35, 1])


In [22]:
from math import sqrt
pred = y_pred_complete[x_test.cpu().numpy()==-1]
ground_truth = y_test[x_test.cpu().numpy()==-1]
print("Test RMSE of STGCN-DAE is: "+ str(sqrt(torch.mean((pred-ground_truth)**2))))
print("Test MAE is of STGCN-DAE is: "+ str(torch.mean(abs(pred-ground_truth))))

Test RMSE of STGCN-DAE is: 4.673917496141987
Test MAE is of STGCN-DAE is: tensor(1.8913, device='cuda:0', grad_fn=<MeanBackward0>)


#Baseline
Only four baselines are here. Scripts for baselines MIDA and LRTC-TNN are ran separately.

LI

In [23]:
#Linear Interpolation
corrupted_test_x[corrupted_test_x==-1] = np.nan
test = pd.DataFrame(corrupted_test_x.numpy())
LI_imputed = test.interpolate(method ='linear', limit_direction ='forward')
#LI_imputed = LI_imputed.dropna()
LI_imputed = LI_imputed.fillna(LI_imputed.mean())
LI_imputed = torch.tensor(LI_imputed.values)
LI_pred = LI_imputed[~mask[86400:103680,]]
print("Test RMSE of Linear Interpolation is: "+ str(sqrt(torch.mean((LI_pred.cpu()-ground_truth.cpu())**2))))
print("Test MAE of Linear Interpolation is: "+ str(torch.mean(abs(LI_pred.cpu()-ground_truth.cpu()))))

Test RMSE of Linear Interpolation is: 19.225026763381486
Test MAE of Linear Interpolation is: tensor(14.9283)


Mean

In [24]:
#Mean Imputation
from math import sqrt
corrupted_test_x[corrupted_test_x==-1] = np.nan
test = pd.DataFrame(corrupted_test_x.numpy())
Mean_imputed = test.fillna(test.mean())
Mean_imputed = torch.tensor(Mean_imputed.values)
Mean_pred = Mean_imputed[~mask[86400:103680,]]
print("Test RMSE of Mean Imputation is: "+ str(sqrt(torch.mean((Mean_pred.cpu()-ground_truth.cpu())**2))))
print("Test MAE of Mean Imputation is: "+ str(torch.mean(abs(Mean_pred.cpu()-ground_truth.cpu()))))

Test RMSE of Mean Imputation is: 28.15015910291683
Test MAE of Mean Imputation is: tensor(26.4180)


MICE

In [ ]:
#MICE
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor()
imp = IterativeImputer(estimator = knn, max_iter = 1, initial_strategy = 'median', imputation_order='ascending',random_state=42)
corrupted_train_x[corrupted_train_x==-1] = np.nan
train = pd.DataFrame(corrupted_train_x.numpy())
imp.fit(train)
corrupted_test_x[corrupted_test_x==-1] = np.nan
test = pd.DataFrame(corrupted_test_x.numpy())
MICE_imputed = imp.transform(test)
MICE_imputed = pd.DataFrame(MICE_imputed)
MICE_imputed = torch.tensor(MICE_imputed.values)
MICE_pred = MICE_imputed[~mask[86400:103680,]]
ground_truth = y_test[x_test.cpu().numpy()==-1]
print("Test RMSE of MICE Interpolation is: "+ str(sqrt(torch.mean((MICE_pred.cpu()-ground_truth.cpu())**2))))
print("Test MAE of MICE Interpolation is: "+ str(torch.mean(abs(MICE_pred.cpu()-ground_truth.cpu()))))

KNN

In [25]:
# KNN Imputation
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=10)
corrupted_train_x[corrupted_train_x==-1] = np.nan
train = pd.DataFrame(corrupted_train_x.numpy())
imputer.fit(train)
KNN_imputed = imputer.transform(test)
KNN_imputed = pd.DataFrame(KNN_imputed)
KNN_imputed = torch.tensor(KNN_imputed.values)
KNN_pred = KNN_imputed[~mask[86400:103680,]]
print("Test RMSE of KNN Interpolation is: "+ str(sqrt(torch.mean((KNN_pred.cpu()-ground_truth.cpu())**2))))
print("Test MAE of KNN Interpolation is: "+ str(torch.mean(abs(KNN_pred.cpu()-ground_truth.cpu()))))

Test RMSE of KNN Interpolation is: 11.31404432008966
Test MAE of KNN Interpolation is: tensor(5.6011)
